## BP Weekly Reporting
### Data Wrangling

Wrangle and repackage weekly data extracts from GA, Facebook and Google AdWords into a tidy dataset prepped for analysis.

In [2]:
# import packages
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from time import strptime
import datetime as dt
import os
import re

In [4]:
## read in FB export
Facebook = pd.read_csv('Bluprint_Weekly_Reporting_Template_Facebook.csv')
print(Facebook.info())
Facebook.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6016 entries, 0 to 6015
Data columns (total 13 columns):
Ad Set ID               6016 non-null float64
Ad Set Name             6016 non-null object
Ad ID                   6016 non-null float64
Ad Name                 6016 non-null object
Day                     6016 non-null object
Amount Spent            6016 non-null float64
Impressions             6016 non-null int64
Link Clicks             2306 non-null float64
3-Second Video Views    2456 non-null float64
Video Watches at 95%    1913 non-null float64
Website Conversions     1625 non-null float64
Reporting Starts        6016 non-null object
Reporting Ends          6016 non-null object
dtypes: float64(7), int64(1), object(5)
memory usage: 611.1+ KB
None


,Ad Set ID,Ad Set Name,Ad ID,Ad Name,Day,Amount Spent,Impressions,Link Clicks,3-Second Video Views,Video Watches at 95%,Website Conversions,Reporting Starts,Reporting Ends
0,2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",9/20/18,179.57,21936,7.0,9513.0,4288.0,4.0,9/20/18,9/20/18
1,2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",9/19/18,328.73,42451,12.0,18673.0,8303.0,3.0,9/19/18,9/19/18
2,2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",9/18/18,318.40,47553,9.0,21720.0,10260.0,6.0,9/18/18,9/18/18
3,2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",2.384300e+16,"Post: ""Talk about puppy love! As if we could a...",9/17/18,173.30,26745,3.0,11966.0,5079.0,3.0,9/17/18,9/17/18
4,2.384300e+16,"Post: ""Talk about MAJOR fashion inspiration. I...",2.384300e+16,"Post: ""Talk about MAJOR fashion inspiration. I...",9/19/18,308.33,25420,974.0,NaN,NaN,1019.0,9/19/18,9/19/18


In [5]:
# Drop unneeded columns 
Facebook.drop(columns=['Ad Set ID', 'Ad ID', 'Reporting Starts', 'Reporting Ends'], axis=1, inplace=True)
Facebook.head()

,Ad Set Name,Ad Name,Day,Amount Spent,Impressions,Link Clicks,3-Second Video Views,Video Watches at 95%,Website Conversions
0,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/20/18,179.57,21936,7.0,9513.0,4288.0,4.0
1,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/19/18,328.73,42451,12.0,18673.0,8303.0,3.0
2,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/18/18,318.40,47553,9.0,21720.0,10260.0,6.0
3,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/17/18,173.30,26745,3.0,11966.0,5079.0,3.0
4,"Post: ""Talk about MAJOR fashion inspiration. I...","Post: ""Talk about MAJOR fashion inspiration. I...",9/19/18,308.33,25420,974.0,NaN,NaN,1019.0


In [6]:
# Change column names 
Facebook.columns = ['Placement', 'Creative', 'Date', 'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions']
Facebook.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/20/18,179.57,21936,7.0,9513.0,4288.0,4.0
1,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/19/18,328.73,42451,12.0,18673.0,8303.0,3.0
2,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/18/18,318.40,47553,9.0,21720.0,10260.0,6.0
3,"Post: ""Talk about puppy love! As if we could a...","Post: ""Talk about puppy love! As if we could a...",9/17/18,173.30,26745,3.0,11966.0,5079.0,3.0
4,"Post: ""Talk about MAJOR fashion inspiration. I...","Post: ""Talk about MAJOR fashion inspiration. I...",9/19/18,308.33,25420,974.0,NaN,NaN,1019.0


In [7]:
Facebook['Placement'].unique()

array(['Post: "Talk about puppy love! As if we could adore cakes..."',
       'Post: "Talk about MAJOR fashion inspiration. In her show..."',
       'Post: "Sundays are for stitching! Spend your crafternoon..."',
       'Post: "Don\'t let all those delicious layers fool you,..."',
       'Conv_SP_FB_TRIAL_Stitch_Sew_EG_WCA_Feed_20180717_112055_',
       'Conv_SP_FB_TRIAL_Stitch_Knit_EG_WCA_Feed_20180717_112050',
       'Conv_SP_FB_TRIAL_Stitch_Knit_BegKnit_LAL_Feed_20180822_112027_3%BegKnitPurchasers',
       'Conv_SP_FB_TRIAL_Savor_Cook_EG_WCA_Feed_20180717_112038_',
       'Conv_SP_FB_TRIAL_Savor_Cake_EG_WCA_Feed_20180717_112037_',
       'Conv_SP_FB_TRIAL_Move_Yoga_EG_WCA_Feed_20180717_112057',
       'Conv_SP_FB_TRIAL_Move_Dance_EG_WCA_Feed_20180717_112039_',
       'Conv_SP_FB_TRIAL_Make_Wood_EG_WCA_Feed_20180717_112056',
       'Conv_SP_FB_TRIAL_Dwell_Home_EG_WCA_Feed_20180717_112048_',
       'Conv_SP_FB_TRIAL_Compose_Paint_EG_WCA_Feed_20180717_112051',
       'Conv_SP_FB_TRIAL_

In [8]:
# read in Google export
Google = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google.csv')
Google.head()

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Bluprint Weekly Reporting Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,"September 17, 2018 - September 23, 2018",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
2,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/21/18,NaN,0,0,0,0,--,0
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/18/18,NaN,0,0,0,0,--,0
4,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/20/18,NaN,0,0,0,0,--,0


In [9]:
# bring header up from row index 1 and drop imported header and row index 0
Google.drop(Google.index[:1], inplace=True)
new_header = Google.iloc[0]
Google = Google[1:]
Google.columns = new_header
Google = Google.reset_index(drop=True)
Google.head()

1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/21/18,NaN,0,0,0,0,--,0
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/18/18,NaN,0,0,0,0,--,0
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/20/18,NaN,0,0,0,0,--,0
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/23/18,NaN,0,0,0,0,--,0
4,AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9/18/18,NaN,0,0,0,0,--,0


In [10]:
# filter out unwanted columns
Google = Google.filter(['Ad group','Ad','Day','Cost','Impressions','Clicks','Views','Video played to 100%','Conversions'])
Google.head()

1,Ad group,Ad,Day,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,Cook_Video_Cook_Video_ScottConant_15_,9/21/18,0,0,0,0,--,0
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/18/18,0,0,0,0,--,0
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/20/18,0,0,0,0,--,0
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/23/18,0,0,0,0,--,0
4,AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View...,All_Awareness_Launch_TVC_:15,9/18/18,0,0,0,0,--,0


In [11]:
# rename columns
Google.columns = ['Placement','Creative','Date','Cost','Impressions','Clicks','Views','Completed Views','Conversions']
Google.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,Cook_Video_Cook_Video_ScottConant_15_,9/21/18,0,0,0,0,--,0
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/18/18,0,0,0,0,--,0
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/20/18,0,0,0,0,--,0
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/23/18,0,0,0,0,--,0
4,AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View...,All_Awareness_Launch_TVC_:15,9/18/18,0,0,0,0,--,0


In [12]:
# create function to strip non-numeric characters from a column
def strp_nonnum(col):
    '''strip string column of non numeric characters, keeping only digits and decimals'''
    col = col.str.extract('([\d]+.?\d*)')
    return col

# strip certain columns of non-numeric characters
cols = ['Cost','Impressions','Clicks','Views','Completed Views','Conversions']
for col in cols:
    Google[col] = strp_nonnum(Google[col])
    Google[col] = Google[col].str.replace(',','').astype(float)
    
Google.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,Cook_Video_Cook_Video_ScottConant_15_,9/21/18,0.0,0.0,0.0,0.0,NaN,0.0
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/20/18,0.0,0.0,0.0,0.0,NaN,0.0
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/23/18,0.0,0.0,0.0,0.0,NaN,0.0
4,AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View...,All_Awareness_Launch_TVC_:15,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0


In [13]:
# read in Google Analytics export
GoogleAnalytics = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google_Analytics_Raw.csv')
GoogleAnalytics.head(10)

,# ----------------------------------------,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,# A-01 Bluprint Web Prod - All,NaN,NaN,NaN,NaN,NaN,NaN
1,# LP Visitor Source_Weekly Reporting Pull,NaN,NaN,NaN,NaN,NaN,NaN
2,# 20180917-20180923,NaN,NaN,NaN,NaN,NaN,NaN
3,# ----------------------------------------,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Marketing Action Id,Date,Sessions,New Users,Bounces,Register (Goal 6 Completions),Subscribe (Goal 1 Completions)
6,113938,20180921,"11,300","9,668","7,997",400,16
7,113938,20180920,"11,248","9,585","8,350",432,16
8,114446,20180919,"10,747","7,682","9,760",2,34
9,114449,20180919,"6,762","5,102","6,219",0,13


In [14]:
# bring up row index 5 as header and drop row indices 0-4
GoogleAnalytics.drop(GoogleAnalytics.index[:5], inplace=True)
new_header = GoogleAnalytics.iloc[0]
GoogleAnalytics = GoogleAnalytics[1:]
GoogleAnalytics.columns = new_header
GoogleAnalytics = GoogleAnalytics.reset_index(drop=True)
GoogleAnalytics.head()

5,Marketing Action Id,Date,Sessions,New Users,Bounces,Register (Goal 6 Completions),Subscribe (Goal 1 Completions)
0,113938,20180921,"11,300","9,668","7,997",400,16
1,113938,20180920,"11,248","9,585","8,350",432,16
2,114446,20180919,"10,747","7,682","9,760",2,34
3,114449,20180919,"6,762","5,102","6,219",0,13
4,112071,20180919,"6,388","4,280","5,180",47,1


In [15]:
GoogleAnalytics.columns = ['MAID','Date','Sessions','New Users','Bounces','Registrations','Subscriptions']
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID      Date Sessions New Users Bounces Registrations Subscriptions
0  113938  20180921   11,300     9,668   7,997           400            16
1  113938  20180920   11,248     9,585   8,350           432            16
2  114446  20180919   10,747     7,682   9,760             2            34
3  114449  20180919    6,762     5,102   6,219             0            13
4  112071  20180919    6,388     4,280   5,180            47             1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2969 entries, 0 to 2968
Data columns (total 7 columns):
MAID             2969 non-null object
Date             2969 non-null object
Sessions         2969 non-null object
New Users        2969 non-null object
Bounces          2969 non-null object
Registrations    2969 non-null object
Subscriptions    2969 non-null object
dtypes: object(7)
memory usage: 162.4+ KB


In [16]:
# bring date column to datetime dtype
GoogleAnalytics['Date'] = pd.to_datetime(GoogleAnalytics['Date'], format = '%Y/%m/%d')

# strip non-numeric characters from certain columns
cols = ['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions']
for col in cols:
    GoogleAnalytics[col] = strp_nonnum(GoogleAnalytics[col])
    GoogleAnalytics[col] = GoogleAnalytics[col].str.replace(',','')
    GoogleAnalytics[col] = pd.to_numeric(GoogleAnalytics[col], errors='coerce')
    
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,113938,2018-09-21,11300,9668,7997,400,16
1,113938,2018-09-20,11248,9585,8350,432,16
2,114446,2018-09-19,10747,7682,9760,2,34
3,114449,2018-09-19,6762,5102,6219,0,13
4,112071,2018-09-19,6388,4280,5180,47,1


In [17]:
# aggregate by MAID and Date, summing remaining columns
GoogleAnalytics = GoogleAnalytics.groupby(['MAID','Date']).sum().reset_index()
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,1.13938E+11,2018-09-17,3,3,3,0,0
1,1.13938E+11,2018-09-19,1,1,0,0,0
2,1.13938E+11,2018-09-20,1,1,1,0,0
3,1.13938E+11,2018-09-21,3,3,3,0,0
4,1.13938E+11,2018-09-22,11,9,10,0,0


In [18]:
##MERGE DATA SETS
all_data = pd.concat([Google, Facebook], ignore_index=True)
all_data.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,Cook_Video_Cook_Video_ScottConant_15_,9/21/18,0.0,0.0,0.0,0.0,NaN,0.0
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/20/18,0.0,0.0,0.0,0.0,NaN,0.0
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/23/18,0.0,0.0,0.0,0.0,NaN,0.0
4,AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View...,All_Awareness_Launch_TVC_:15,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0


In [19]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103188 entries, 0 to 103187
Data columns (total 9 columns):
Placement          64074 non-null object
Creative           64074 non-null object
Date               64074 non-null object
Cost               38782 non-null float64
Impressions        64074 non-null float64
Clicks             60364 non-null float64
Views              35222 non-null float64
Completed Views    1920 non-null float64
Conversions        34391 non-null float64
dtypes: float64(6), object(3)
memory usage: 7.1+ MB


In [20]:
# create new columns and categorize
all_data['MAID'] = all_data['Placement'].str.extract('(_1\d\d\d\d\d)', expand=True)
all_data['MAID'] = all_data['MAID'].str.strip('_')
all_data['Funnel'] = all_data['Placement'].str.split('_').str[0]
#all_data['Tactic'] = all_data['Placement'].str.split('_').str[1]
all_data['Publisher'] = all_data['Placement'].str.split('_').str[2]
all_data['Hook'] = all_data['Placement'].str.split('_').str[3]
all_data['Mega'] = all_data['Placement'].str.split('_').str[4]
all_data['Micro'] = all_data['Placement'].str.split('_').str[5]
all_data['Campaign'] = all_data['Placement'].str.split('_').str[6]
all_data['Targ/aud/pin'] = all_data['Placement'].str.split('_').str[7]
all_data['Keyword'] = all_data['Placement'].str.split('_').str[8]
all_data.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword
0,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,Cook_Video_Cook_Video_ScottConant_15_,9/21/18,0.0,0.0,0.0,0.0,NaN,0.0,112067,Cons,YT,SERIESBLUPRINT,Site Visitors,TBD,-,SV,Tru-View
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0,112063,AW,YT,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/20/18,0.0,0.0,0.0,0.0,NaN,0.0,112063,AW,YT,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/23/18,0.0,0.0,0.0,0.0,NaN,0.0,112063,AW,YT,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View
4,AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View...,All_Awareness_Launch_TVC_:15,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0,112062,AW,YT,MULTISHOWCASE,Move,TBD,-,im,Tru-View


In [21]:
all_data['Mega'].unique()

array(['Site Visitors', 'Savor', 'Move', 'Dwell', 'Compose', 'Make',
       'Engagement', 'Other', 'Lookalike Audience', 'Together', 'Stitch',
       'VV', 'Craftsy LAL', 'Craftsy SV', 'All', 'BP CA', nan],
      dtype=object)

In [22]:
# rename categories for Funnel and Publisher columns
all_data['Funnel'] = all_data['Funnel'].replace({'AW' : 'Awareness', 'Cons' : 'Consideration', 'CONS' : 'Consideration', 'Conv' : 'Conversion', 'CONV' : 'Conversion'})
all_data['Publisher'] = all_data['Publisher'].replace({'G' : 'Google Search', 'GDN' : 'Google Display Network', 'YT' : 'YouTube', 'FB' : 'Facebook'})
all_data['Mega'] = all_data['Mega'].replace({'Site Visitors':'All/Other','Engagement':'All/Other','Other':'All/Other','Lookalike Audience':'All/Other','VV':'All/Other','Craftsy LAL':'All/Other','Craftsy SV':'All/Other','All':'All/Other','BP CA':'All/Other'})
print(all_data['Funnel'].unique())
print(all_data['Publisher'].unique())
all_data['Mega'].unique()

['Consideration' 'Awareness' 'Conversion' nan
 'Post: "Talk about puppy love! As if we could adore cakes..."'
 'Post: "Talk about MAJOR fashion inspiration. In her show..."'
 'Post: "Sundays are for stitching! Spend your crafternoon..."'
 'Post: "Don\'t let all those delicious layers fool you,..."'
 'BST | Make a Connection | Parents | LPV |114525'
 'BST | Make a Connection | Not Parents | LPV |114525']
['YouTube' 'Google Display Network' 'Google Search' nan 'Facebook']


array(['All/Other', 'Savor', 'Move', 'Dwell', 'Compose', 'Make',
       'Together', 'Stitch', nan], dtype=object)

In [23]:
all_data.loc[all_data['Publisher'] == 'Google Search', 'Cost'].sum()

27066.519999999997

In [24]:
all_data['Placement'].unique()

array(['Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-_SV_Tru-View_20180717_TBD_dwell_112067',
       'AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-View_20180717_TBD_cooking enthusiast_112063',
       'AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View_20180717_TBD_outdoor & rec items_112062',
       'AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_im_Tru-View_20180717_TBD_home appliances_112063',
       'Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-_SV_Tru-View_20180717_TBD_make_112068',
       'AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_aa_Tru-View_20180717_TBD_travel_112062',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_IM__Res_20180717_-HomeFurnishing_112073',
       'AW_OLV_YT_MULTISHOWCASE_Compose_TBD_-_im_Tru-View_20180717_TBD_paint_112059',
       'AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-View_20180717_TBD_foodies_112063',
       'AW_OLV_YT_MULTISHOWCASE_Make_TBD_-_im_Tru-View_20180717_TBD_beauty products & services_112061',
       'AW_OLV_YT_MULTISHOWCASE_Make_TBD_-_aa_Tru-View_20180717_TBD_fashioni

In [63]:
all_data.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword
0,Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-...,Cook_Video_Cook_Video_ScottConant_15_,9/21/18,0.0,0.0,0.0,0.0,NaN,0.0,112067,Consideration,YouTube,SERIESBLUPRINT,All/Other,TBD,-,SV,Tru-View
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0,112063,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/20/18,0.0,0.0,0.0,0.0,NaN,0.0,112063,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,9/23/18,0.0,0.0,0.0,0.0,NaN,0.0,112063,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View
4,AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View...,All_Awareness_Launch_TVC_:15,9/18/18,0.0,0.0,0.0,0.0,NaN,0.0,112062,Awareness,YouTube,MULTISHOWCASE,Move,TBD,-,im,Tru-View


In [25]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103188 entries, 0 to 103187
Data columns (total 18 columns):
Placement          64074 non-null object
Creative           64074 non-null object
Date               64074 non-null object
Cost               38782 non-null float64
Impressions        64074 non-null float64
Clicks             60364 non-null float64
Views              35222 non-null float64
Completed Views    1920 non-null float64
Conversions        34391 non-null float64
MAID               63252 non-null object
Funnel             64074 non-null object
Publisher          64036 non-null object
Hook               64036 non-null object
Mega               64036 non-null object
Micro              64036 non-null object
Campaign           64036 non-null object
Targ/aud/pin       64036 non-null object
Keyword            64036 non-null object
dtypes: float64(6), object(12)
memory usage: 14.2+ MB


In [26]:
all_data['Placement'].unique()

array(['Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-_SV_Tru-View_20180717_TBD_dwell_112067',
       'AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-View_20180717_TBD_cooking enthusiast_112063',
       'AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_im_Tru-View_20180717_TBD_outdoor & rec items_112062',
       'AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_im_Tru-View_20180717_TBD_home appliances_112063',
       'Cons_OLV_YT_SERIESBLUPRINT_Site Visitors_TBD_-_SV_Tru-View_20180717_TBD_make_112068',
       'AW_OLV_YT_MULTISHOWCASE_Move_TBD_-_aa_Tru-View_20180717_TBD_travel_112062',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_IM__Res_20180717_-HomeFurnishing_112073',
       'AW_OLV_YT_MULTISHOWCASE_Compose_TBD_-_im_Tru-View_20180717_TBD_paint_112059',
       'AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-View_20180717_TBD_foodies_112063',
       'AW_OLV_YT_MULTISHOWCASE_Make_TBD_-_im_Tru-View_20180717_TBD_beauty products & services_112061',
       'AW_OLV_YT_MULTISHOWCASE_Make_TBD_-_aa_Tru-View_20180717_TBD_fashioni

In [27]:
# calculate daily impressions by MAID
imp_by_MAID = all_data.groupby(by=['MAID', 'Date'])['Impressions'].sum().reset_index()
imp_by_MAID.head()

,MAID,Date,Impressions
0,111868,9/17/18,132.0
1,111868,9/18/18,119.0
2,111868,9/19/18,134.0
3,111868,9/20/18,130.0
4,111868,9/21/18,105.0


In [28]:
imp_by_MAID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 3 columns):
MAID           1169 non-null object
Date           1169 non-null object
Impressions    1169 non-null float64
dtypes: float64(1), object(2)
memory usage: 27.5+ KB


In [29]:
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,1.13938E+11,2018-09-17,3,3,3,0,0
1,1.13938E+11,2018-09-19,1,1,0,0,0
2,1.13938E+11,2018-09-20,1,1,1,0,0
3,1.13938E+11,2018-09-21,3,3,3,0,0
4,1.13938E+11,2018-09-22,11,9,10,0,0


In [30]:
GoogleAnalytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2969 entries, 0 to 2968
Data columns (total 7 columns):
MAID             2969 non-null object
Date             2969 non-null datetime64[ns]
Sessions         2969 non-null int64
New Users        2969 non-null int64
Bounces          2969 non-null int64
Registrations    2969 non-null int64
Subscriptions    2969 non-null int64
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 162.4+ KB


In [31]:
# merge imp_by_MAID and GoogleAnalytics
imp_by_MAID['Date'] = pd.to_datetime(imp_by_MAID['Date'])
GoogleAnalytics = pd.merge(GoogleAnalytics, imp_by_MAID, how='inner', on=['MAID', 'Date'])
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID       Date  Sessions  New Users  Bounces  Registrations  \
0  111868 2018-09-17        58         44       28              1   
1  111868 2018-09-18        47         38       22              0   
2  111868 2018-09-19        64         45       30              2   
3  111868 2018-09-20        70         46       38              2   
4  111868 2018-09-21        52         34       26              2   

   Subscriptions  Impressions  
0              0        132.0  
1              1        119.0  
2              0        134.0  
3              1        130.0  
4              1        105.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 972 entries, 0 to 971
Data columns (total 8 columns):
MAID             972 non-null object
Date             972 non-null datetime64[ns]
Sessions         972 non-null int64
New Users        972 non-null int64
Bounces          972 non-null int64
Registrations    972 non-null int64
Subscriptions    972 non-null int64
Impressions      972 non-nu

In [32]:
##CALCULATE COLUMNS TO BE JOINED INTO OVERALL DATASET
GoogleAnalytics['Session Rate'] = GoogleAnalytics['Sessions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['New Session Rate'] = GoogleAnalytics['New Users'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Bounce Rate'] = GoogleAnalytics['Bounces'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Reg Rate'] = GoogleAnalytics['Registrations'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Sub Rate'] = GoogleAnalytics['Subscriptions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalyticsRates = GoogleAnalytics
GoogleAnalyticsRates.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions,Impressions,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-09-17,58,44,28,1,0,132.0,0.439394,0.758621,0.482759,0.007576,0.000000
1,111868,2018-09-18,47,38,22,0,1,119.0,0.394958,0.808511,0.468085,0.000000,0.008403
2,111868,2018-09-19,64,45,30,2,0,134.0,0.477612,0.703125,0.468750,0.014925,0.000000
3,111868,2018-09-20,70,46,38,2,1,130.0,0.538462,0.657143,0.542857,0.015385,0.007692
4,111868,2018-09-21,52,34,26,2,1,105.0,0.495238,0.653846,0.500000,0.019048,0.009524


In [33]:
GoogleAnalyticsRates.drop(columns=['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions', 'Impressions'], axis=1, inplace=True)
GoogleAnalyticsRates.head()

,MAID,Date,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-09-17,0.439394,0.758621,0.482759,0.007576,0.000000
1,111868,2018-09-18,0.394958,0.808511,0.468085,0.000000,0.008403
2,111868,2018-09-19,0.477612,0.703125,0.468750,0.014925,0.000000
3,111868,2018-09-20,0.538462,0.657143,0.542857,0.015385,0.007692
4,111868,2018-09-21,0.495238,0.653846,0.500000,0.019048,0.009524


In [34]:
##JOIN NEW GOOGLE ANALYTICS TO ALL_DATA_MERGE
all_data['Date'] = pd.to_datetime(all_data['Date'])
final_data_merge = pd.merge(all_data, GoogleAnalyticsRates, how='inner', on=['MAID', 'Date'])
print(final_data_merge.info())
final_data_merge.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58662 entries, 0 to 58661
Data columns (total 23 columns):
Placement           58662 non-null object
Creative            58662 non-null object
Date                58662 non-null datetime64[ns]
Cost                36285 non-null float64
Impressions         58662 non-null float64
Clicks              54992 non-null float64
Views               32768 non-null float64
Completed Views     1887 non-null float64
Conversions         31938 non-null float64
MAID                58662 non-null object
Funnel              58662 non-null object
Publisher           58662 non-null object
Hook                58662 non-null object
Mega                58662 non-null object
Micro               58662 non-null object
Campaign            58662 non-null object
Targ/aud/pin        58662 non-null object
Keyword             58662 non-null object
Session Rate        58662 non-null float64
New Session Rate    58662 non-null float64
Bounce Rate         58662 non-null f

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,aa,Tru-View,0.0,0.0,1.0,NaN,NaN
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_im_Tru-Vie...,All_Awareness_Launch_TVC_:15,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,im,Tru-View,0.0,0.0,1.0,NaN,NaN
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,aa,Tru-View,0.0,0.0,1.0,NaN,NaN
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_AA_Tru-Vie...,All_Awareness_Launch_TVC_:15,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,AA,Tru-View,0.0,0.0,1.0,NaN,NaN
4,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_im_Tru-Vie...,All_Awareness_Launch_TVC_:30,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,im,Tru-View,0.0,0.0,1.0,NaN,NaN


In [35]:
# create new columns by multiplying others
final_data_merge['Sessions'] = final_data_merge['Impressions'].mul(final_data_merge['Session Rate'])
final_data_merge['New Sessions'] = final_data_merge['Sessions'].mul(final_data_merge['New Session Rate'])
final_data_merge['Bounces'] = final_data_merge['Sessions'].mul(final_data_merge['Bounce Rate'])
final_data_merge['Registrations'] = final_data_merge['Impressions'].mul(final_data_merge['Reg Rate'])
final_data_merge['Subscriptions'] = final_data_merge['Impressions'].mul(final_data_merge['Sub Rate'])
final_data_merge.drop(columns=['Session Rate', 'New Session Rate', 'Bounce Rate', 'Reg Rate', 'Sub Rate'], axis=1, inplace=True)
final_data_merge.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,aa,Tru-View,0.0,0.0,0.0,NaN,NaN
1,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_im_Tru-Vie...,All_Awareness_Launch_TVC_:15,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,im,Tru-View,0.0,0.0,0.0,NaN,NaN
2,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_aa_Tru-Vie...,All_Awareness_Launch_TVC_:30,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,aa,Tru-View,0.0,0.0,0.0,NaN,NaN
3,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_AA_Tru-Vie...,All_Awareness_Launch_TVC_:15,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,AA,Tru-View,0.0,0.0,0.0,NaN,NaN
4,AW_OLV_YT_MULTISHOWCASE_Savor_TBD_-_im_Tru-Vie...,All_Awareness_Launch_TVC_:30,2018-09-18,0.0,0.0,0.0,0.0,NaN,0.0,112063,...,Savor,TBD,-,im,Tru-View,0.0,0.0,0.0,NaN,NaN


In [36]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58662 entries, 0 to 58661
Data columns (total 23 columns):
Placement          58662 non-null object
Creative           58662 non-null object
Date               58662 non-null datetime64[ns]
Cost               36285 non-null float64
Impressions        58662 non-null float64
Clicks             54992 non-null float64
Views              32768 non-null float64
Completed Views    1887 non-null float64
Conversions        31938 non-null float64
MAID               58662 non-null object
Funnel             58662 non-null object
Publisher          58662 non-null object
Hook               58662 non-null object
Mega               58662 non-null object
Micro              58662 non-null object
Campaign           58662 non-null object
Targ/aud/pin       58662 non-null object
Keyword            58662 non-null object
Sessions           58662 non-null float64
New Sessions       58662 non-null float64
Bounces            58662 non-null float64
Registrations 

In [37]:
##ADD REMAINING COLUMNS
final_data_merge['Start Date'] = final_data_merge.groupby(['Placement'])['Date'].transform(min)
final_data_merge['Video v Static'] = np.where(final_data_merge['Views']>=1, 'Video', 'Static')
final_data_merge['Impressions with Video View'] = np.where(final_data_merge['Views']>=1, final_data_merge['Impressions'], 0)
final_data_merge['Reporting Month'] = final_data_merge['Date'].dt.strftime('%B')
final_data_merge['Week Start'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)
final_data_merge['Reporting Period'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)

In [38]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58662 entries, 0 to 58661
Data columns (total 29 columns):
Placement                      58662 non-null object
Creative                       58662 non-null object
Date                           58662 non-null datetime64[ns]
Cost                           36285 non-null float64
Impressions                    58662 non-null float64
Clicks                         54992 non-null float64
Views                          32768 non-null float64
Completed Views                1887 non-null float64
Conversions                    31938 non-null float64
MAID                           58662 non-null object
Funnel                         58662 non-null object
Publisher                      58662 non-null object
Hook                           58662 non-null object
Mega                           58662 non-null object
Micro                          58662 non-null object
Campaign                       58662 non-null object
Targ/aud/pin                  

In [39]:
#REORGANIZE REMAINING COLUMNS
final_data_merge = final_data_merge[['Funnel', 'Publisher', 'Hook', 'Mega', 'Micro', 
                                     'Campaign', 'Targ/aud/pin', 'Keyword', 'Start Date', 'MAID', 
                                     'Video v Static', 'Impressions with Video View', 'Week Start', 
                                     'Reporting Month', 'Reporting Period', 'Placement', 'Creative', 'Date', 
                                     'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions', 
                                     'Sessions', 'New Sessions', 'Bounces', 'Registrations', 'Subscriptions']]
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58662 entries, 0 to 58661
Data columns (total 29 columns):
Funnel                         58662 non-null object
Publisher                      58662 non-null object
Hook                           58662 non-null object
Mega                           58662 non-null object
Micro                          58662 non-null object
Campaign                       58662 non-null object
Targ/aud/pin                   58662 non-null object
Keyword                        58662 non-null object
Start Date                     58662 non-null datetime64[ns]
MAID                           58662 non-null object
Video v Static                 58662 non-null object
Impressions with Video View    58662 non-null float64
Week Start                     58662 non-null datetime64[ns]
Reporting Month                58662 non-null object
Reporting Period               58662 non-null datetime64[ns]
Placement                      58662 non-null object
Creative          

In [40]:
final_data_merge['Date'].unique()

array(['2018-09-18T00:00:00.000000000', '2018-09-17T00:00:00.000000000',
       '2018-09-21T00:00:00.000000000', '2018-09-23T00:00:00.000000000',
       '2018-09-19T00:00:00.000000000', '2018-09-22T00:00:00.000000000',
       '2018-09-20T00:00:00.000000000'], dtype='datetime64[ns]')

In [41]:
final_data_merge['Targ/aud/pin'].unique()

array(['aa', 'im', 'AA', 'VV', 'SV', 'SV LAL', 'vv', 'LAL', 'IM', 'EG',
       'JoshuaJohnRussell', 'E', 'FabioViviani', 'Celebrity', 'BMM',
       'JessicaLong', 'Celebrrity', 'Generic', 'JoshuaVergara',
       'BrookeLark', 'Doodle-Wars', 'CA', 'BPRM', 'RM', 'Accessories',
       'WCA', 'Interest', 'BluprintLAL', 'RT', 'Demo'], dtype=object)

In [78]:
final_data_merge.to_csv('weekly_data_merge.csv')

### Still To Do

- output no index column in final_data_merge

✅ drop 'Tactic' column 

✅ rename 'Video Completion' to 'Completed Views'

- audit columns for:
    - data type
    - quality
    - etc.
    
✅ Refine 'Mega' column to only include unique values, not duplicates with slight variations OR assign "All/Other" properly

In [79]:
final_data_merge.head()

,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Start Date,MAID,...,Impressions,Clicks,Views,Completed Views,Conversions,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View,2018-09-17,112063,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
1,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,im,Tru-View,2018-09-17,112063,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
2,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,aa,Tru-View,2018-09-17,112063,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
3,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,AA,Tru-View,2018-09-17,112063,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
4,Awareness,YouTube,MULTISHOWCASE,Savor,TBD,-,im,Tru-View,2018-09-17,112063,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
